In [2]:
import numpy as np
import argparse
import time
import cv2
import os

In [3]:
ssd_weights=os.path.join('model','frozen_inference_graph.pb')
ssd_pbtxt=os.path.join('model','ssd_mobilenet.pbtxt')

In [4]:
classNames = {0: 'background',
              1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
              7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
              13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
              18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
              24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
              32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
              37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
              41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
              46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
              51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
              56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
              61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
              67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
              75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
              80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
              86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush'}

In [5]:
np.random.seed(49)
COLORS = np.random.randint(0, 255, size=(90, 3),dtype="uint8")

In [6]:
ssd_model=cv2.dnn.readNetFromTensorflow(ssd_weights,ssd_pbtxt)

In [9]:
cap=cv2.VideoCapture("traffic-mini.mp4")
pressed=False
while True:
    is_frame,frame=cap.read()
    if not is_frame:
        break
    im_height,im_width,garbage=frame.shape
    ssd_model.setInput(cv2.dnn.blobFromImage(frame,size=(300,300),swapRB=True))
    output=ssd_model.forward()
    for detection in output[0,0,:,:]:
        confidence=detection[2]
        if confidence > .5:
            class_id=int(detection[1])
            class_name=classNames[class_id]
            box_x = detection[3] * im_width
            box_y = detection[4] * im_height
            box_width = detection[5] * im_width
            box_height = detection[6] * im_height
            color=[int(x) for x in COLORS[class_id]]
            cv2.rectangle(frame, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color)
            text = "{}: {:.4f}".format(class_name,confidence)
            cv2.putText(frame,text ,(int(box_x), int(box_y-5)),cv2.FONT_HERSHEY_SIMPLEX,0.5,color)
    cv2.imshow('Object Detection SSD', cv2.resize(frame,(1366,768)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        pressed=True
        cv2.destroyAllWindows()
        break
if not pressed:
    cv2.destroyAllWindows()